In [ ]:
#!pip install transformers==4.35.0 sentence-transformers==2.2.2 chromadb==0.4.6 accelerate tqdm 

  Using cached transformers-4.35.0-py3-none-any.whl.metadata (123 kB)
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached chromadb-0.4.6-py3-none-any.whl.metadata (6.8 kB)
  Using cached accelerate-1.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached fastapi-0.99.1-py3-none-any.whl.metadata (23 kB)
  Using cached posthog-3.23.0-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependen

# Creación de un Entorno de Python

En este documento, vamos a explicar cómo configurar un nuevo entorno de Python utilizando tanto `venv` como `conda`.

## Usando venv

`venv` es una herramienta que viene preinstalada con Python (versión 3.3 o superior) y permite crear entornos virtuales de Python aislados.

### Pasos

1. **Crear un entorno virtual:** Abre la terminal y ejecuta el siguiente comando en el directorio donde desees crear el entorno virtual.

   ```bash
   python3 -m venv nombre_del_entorno


2. **Activar el entorno virtual:** Utiliza el siguiente comando para activar el entorno.


  Linux/Mac:

```bash
    source nombre_del_entorno/bin/activate
```


   Windows:

```bash
    .\nombre_del_entorno\Scripts\activate
```


3. **Instalar dependencias desde requirements.txt con pip:** Una vez activado el entorno, puedes instalar todas las dependencias necesarias desde un archivo `requirements.txt` utilizando:

   ```bash
   pip install -r requirements.txt

4. **Desactivar el entorno virtual:** Para salir del entorno virtual, simplemente ejecuta:

```bash
   deactivate

## Usando Conda
conda es un sistema de gestión de paquetes y de entorno que puede instalar paquetes de diferentes lenguajes.

### Pasos
1. **Instalar Conda:** Puedes descargar e instalar Conda desde [este enlace](https://www.anaconda.com/download).

2. **Crear un entorno Conda:** Para crear un nuevo entorno con Conda, abre la terminal y ejecuta:

```bash
    conda create --name nombre_del_entorno python=3.9 ipykernel
```

Para esta biblioteca, vamos a usar python 3.9 y para interactuar con Jupyter Notebooks vamos a usar "ipykernel" o "jupyter"

3. **Activar el entorno Conda:** Utiliza el siguiente comando para activar el entorno.

```bash
    conda activate nombre_del_entorno
```
4. **Instalar dependencias desde requirements.txt con pip:** Al igual que con venv, puedes instalar las dependencias necesarias desde un archivo requirements.txt en el entorno Conda activado utilizando:
```bash
    pip install -r requirements.txt
```

5. Desactivar el entorno Conda: Para salir del entorno Conda, simplemente ejecuta:

```bash
    conda deactivate
```

In [16]:
# 1️⃣ Importar librerías
import openai
import chromadb
from chromadb.config import Settings
import os
import uuid

# Configurar la clave de API de OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")  # Usa una variable de entorno para mayor seguridad

# Función para obtener embeddings usando OpenAI
def get_embeddings(text):
    try:
        response = openai.embeddings.create(
            input=text,
            model="text-embedding-ada-002"  # Modelo de OpenAI para embeddings
        )
        return response.data[0].embedding
    except openai.error.OpenAIError as e:
        print(f"❌ Error al obtener embeddings: {e}")
        return None

# Inicializar ChromaDB con persistencia
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_or_create_collection(name="documentos_ia")

def cargar_documentos(base_path="/Users/fabricio.denardi/Documents/CEIA/BDIA/repos/MIA_01c_BDIA/TP3"):
    docs_path = os.path.join(base_path, "docs","recipes")
    documentos = []
    metadatos = []
    ids = []

    for filename in os.listdir(docs_path):
        if filename.endswith(".md"):
            with open(os.path.join(docs_path, filename), 'r', encoding="utf-8") as f:
                text = f.read().strip()
                chunks = [text[i:i+500] for i in range(0, len(text), 500)]
                
                for i, chunk in enumerate(chunks):
                    documentos.append(chunk)
                    metadatos.append({"source": filename})
                    ids.append(str(uuid.uuid4()))  # ID único
    
    return documentos, metadatos, ids

def crear_base_datos():
    docs, metas, ids = cargar_documentos()
    embeddings = [get_embeddings(d) for d in docs if get_embeddings(d) is not None]
    collection.add(documents=docs, embeddings=embeddings, metadatas=metas, ids=ids)
    print("✅ Base de datos creada con éxito.")

# --- CRUD FUNCIONES ---

def create_example(new_doc):
    """Añadir un documento nuevo"""
    embedding = get_embeddings(new_doc)
    if embedding:
        doc_id = str(uuid.uuid4())
        metadata = {"source": "nuevo_documento.txt"}
        
        collection.add(
            documents=[new_doc],
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[doc_id]
        )
        print(f"✅ Documento añadido con ID: {doc_id}")
    else:
        print("❌ No se pudo añadir el documento debido a un error en los embeddings.")

def read_example(query):
    """Consultar documentos similares"""
    query_emb = get_embeddings(query)
    if query_emb:
        results = collection.query(
            query_embeddings=[query_emb],
            n_results=1
        )
        
        if results["documents"]:
            print(f"📄 Mejor coincidencia: {results['documents'][0][0]}")
            print(f"📁 Fuente: {results['metadatas'][0][0]['source']}")
            print(f"🆔 ID: {results['ids'][0][0]}")
        else:
            print("⚠️ No se encontraron resultados.")
    else:
        print("❌ No se pudo realizar la consulta debido a un error en los embeddings.")

def get_documents_by_id(doc_id):
    """Obtener un documento por ID"""
    results = collection.get(ids=[doc_id], include=["documents", "metadatas", "embeddings"])
    
    if results["documents"]:
        print(f"📄 Documento: {results['documents'][0]}")
        print(f"📁 Fuente: {results['metadatas'][0]['source']}")
    else:
        print("⚠️ Documento no encontrado.")

def update_example(old_id, new_text):
    """Actualizar un documento (eliminar y reinsertar)"""
    collection.delete(ids=[old_id])
    
    new_id = str(uuid.uuid4())
    embedding = get_embeddings(new_text)
    if embedding:
        metadata = {"source": "documento_actualizado.txt"}
        
        collection.add(
            documents=[new_text],
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[new_id]
        )
        print(f"♻️ Documento actualizado. Nuevo ID: {new_id}")
    else:
        print("❌ No se pudo actualizar el documento debido a un error en los embeddings.")

def delete_example(doc_id):
    """Eliminar por ID"""
    collection.delete(ids=[doc_id])
    print(f"🗑️ Documento con ID '{doc_id}' eliminado.")

print('listo')

listo


In [17]:
crear_base_datos()

✅ Base de datos creada con éxito.


In [18]:
get_embeddings("Add all ingredients into blender, blend for 3 minutes or until smooth in texture.")

[-0.003770906012505293,
 -0.006599085405468941,
 -0.023411041125655174,
 -0.00565308565273881,
 0.015083624050021172,
 -0.00786260049790144,
 -0.001582667580805719,
 -0.021014945581555367,
 -0.0038461931981146336,
 -0.03006250038743019,
 0.01929970644414425,
 0.029931565746665,
 -0.01020632404834032,
 -0.02008531242609024,
 -0.010815167799592018,
 0.02229810133576393,
 0.020491208881139755,
 0.015188371762633324,
 0.020936384797096252,
 -0.010396178811788559,
 -0.01953538879752159,
 0.009283237159252167,
 0.010893728584051132,
 -0.015869230031967163,
 -0.028831718489527702,
 0.023803845047950745,
 0.00034063358907587826,
 -0.0022357020061463118,
 0.0002036641089944169,
 0.016811955720186234,
 0.027417629957199097,
 -0.006608905736356974,
 -0.0022242453414946795,
 -0.007024621590971947,
 -0.023306293413043022,
 0.013577880337834358,
 -0.003291359404101968,
 0.0004689082270488143,
 0.020163873210549355,
 -0.00867439340800047,
 0.01640605926513672,
 -0.01846172660589218,
 0.00084534415509

In [19]:
results = collection.get(include=["embeddings", "documents", "metadatas"])

In [20]:
for id, content, metadata, embedding in zip(results["ids"], results["documents"], results["metadatas"], results["embeddings"]):
    print(f"ID: {id}, Contenido: {content[:100]} ", f"Metadata: {metadata}", f"Embedding: {embedding[:5]}...")

ID: 13e72961-a0db-4709-8644-af65868394ea, Contenido: El sol se alzaba sobre las colinas de Buenos Aires, pintando el cielo con tonos naranja y rosa. El a  Metadata: {'source': 'doc1.txt'} Embedding: [0.002886376576498151, 0.0004124291008338332, -0.019652780145406723, -0.027064379304647446, 0.002653928007930517]...
ID: 1304f150-8c1c-4669-b0dc-30c62db3aa86, Contenido: ciosa continuaba, con el tráfico constante y el murmullo de las conversaciones. Los turistas se mara  Metadata: {'source': 'doc1.txt'} Embedding: [-0.011876767501235008, -0.010162802413105965, 0.017784904688596725, -0.010875274427235126, 0.008072437718510628]...
ID: efde7866-808d-4c13-a277-8db4f8bc256d, Contenido:  los restaurantes servían deliciosas parrilladas.

Hacia el norte, el elegante barrio de Recoleta in  Metadata: {'source': 'doc1.txt'} Embedding: [-0.004808980040252209, 6.465915066655725e-05, 0.01111509557813406, -0.008399512618780136, 0.013895489275455475]...
ID: 619ccb53-f5bd-4435-815e-3e21249835e8, Contenido: 

In [21]:
while True:
    opcion = input("Elige una opción o escribe ayuda: ")
    
    if opcion=="ayuda":
        print("\n--- Menú CRUD ---")
        print("1. Crear documento")
        print("2. Consultar por tema")
        print("3. Actualizar documento (por ID)")
        print("4. Eliminar documento (por ID)")
        print("5. Buscar documento (por ID)")
        print("6. Salir")

    if opcion == "1":
        nuevo_doc = input("Ingresa el texto a añadir: ")
        create_example(nuevo_doc)
        
    elif opcion == "2":
        consulta = input("Consulta: ")
        read_example(consulta)
        
    elif opcion == "3":
        doc_id = input("ID del documento a actualizar: ")
        nuevo_texto = input("Nuevo contenido: ")
        update_example(doc_id, nuevo_texto)
        
    elif opcion == "4":
        doc_id = input("ID del documento a eliminar: ")
        delete_example(doc_id)  

    elif opcion == "5":
        doc_id = input("ID del documento a buscar: ")
        get_documents_by_id(doc_id)
        
    elif opcion == "6":
        print("👋 Saliendo del programa.")
        break

👋 Saliendo del programa.


## ¿Cómo sigue esto?

### de forma conceptual debemos:
1. Recuperar el contexto relevante con una busqueda como read_example(query) o similar.
2. Formular el prompt para el llm, juntando la consulta original con los embeddings relevantes.
3. Enviar la Petición al Endpoint (en este caso LM Studio).
4. Recibir y mostrar la respuesta.

In [24]:
# Esta sección es para la integración con LM Studio y no es parte del CRUD. Es solo un ejemplo de cómo se podría hacer una consulta a un modelo de lenguaje como Llama 3 o similar.
# 2️⃣ Integración con LM Studio (Llama 3 o similar)
import requests
import json

LM_STUDIO_URL = "http://127.0.0.1:1234/v1/chat/completions"  

def query_llama(prompt):
    """Envia un prompt a LM Studio y recibe la respuesta."""
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 256, 
        # Es posible agregar otros parámetros de generación aca (temperature, top_p, etc.)
    }
    try:
        response = requests.post(LM_STUDIO_URL, headers=headers, data=json.dumps(data))
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con LM Studio: {e}")
        return None
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error al procesar la respuesta de LM Studio: {e}")
        return None

def responder_pregunta(pregunta):
    """Busca documentos relevantes y luego consulta a Llama."""
    query_emb = get_embeddings(pregunta)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=3  # Obtener algunos documentos relevantes como contexto
    )

    if results["documents"]:
        contexto = "\n\n".join(results["documents"][0])
        prompt = f"Basándote en la siguiente información:\n\n{contexto}\n\nResponde a la siguiente pregunta: {pregunta}" # Juntar el contexto obtenido y la pregunta
        respuesta = query_llama(prompt)
        return respuesta
    else:
        return "No se encontraron documentos relevantes para responder a tu pregunta."

# --- Ejemplo de uso ---
if __name__ == "__main__":
    crear_base_datos() 

    while True:
        pregunta_usuario = input("Pregúntame algo sobre los documentos (o escribe 'salir'): ")
        if pregunta_usuario.lower() == 'salir':
            break

        print(f"🤖 Pregunta: {pregunta_usuario}")
        respuesta_ia = responder_pregunta(pregunta_usuario)
        if respuesta_ia:
            print(f"🤖 Respuesta de la IA: {respuesta_ia}")

✅ Base de datos creada con éxito.
🤖 Pregunta: based on the context, give me the step by step to prepare a bechamel 
🤖 Respuesta de la IA: <think>
Okay, I need to figure out how to make Béchamel sauce based on the information provided in the issue description. Let's see what's given.

First, looking at the ingredients listed:
- 50g butter
- 50g flour
- 500ml milk
- Seasoning (white pepper preferred)
- 100g cheddar cheese
- Nutmeg

The method steps provided are partial. The user started by melting the butter, adding flour and stirring until a dough is formed. Then in step 3, it says "gradually add the," but that's cut off.

I think step 3 might be adding something else. Since there's cheddar cheese mentioned in ingredients, perhaps they meant to add the cheese gradually as well. Alternatively, maybe they forgot part of the method after step 2.

Let me recall how Béchamel is made. Typically, you start by melting butter and flour to make a roux, then slowly add milk while stirring until it